# Pandas 예제 : 미국 출생률에 휴일이 미치는 영향
출생율 데이터 : https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv

- 날짜, 성별로 분류한 출생 수를 담고 있음


In [2]:
import pandas as pd

births = pd.read_csv('births.csv')
births.head()

,year,month,day,gender,births
0,1969,1,1.0,F,4046
1,1969,1,1.0,M,4440
2,1969,1,2.0,F,4454
3,1969,1,2.0,M,4548
4,1969,1,3.0,F,4548


In [17]:
# 연대별 남녀의 출생 수
births['decade'] = 10 * (births['year'] // 10) # 10으로 나누어 일의 자리를 없애줌 최종적으로 ex, 1969년 => 1960년대로 출력

# aggfunc : 어떤 유형의 집계를 적용할지, 디폴트값 : 평균
b_df = births.pivot_table('births', index='decade', columns='gender', aggfunc='sum')
b_df

gender,F,M
decade,,
1960,1753634,1846572
1970,16263075,17121550
1980,18310351,19243452
1990,19479454,20420553
2000,18229309,19106428


In [26]:
import numpy as np

quartiles = np.percentile(births['births'], [25, 50, 75])
mu = quartiles[1]
sig = 0.74 * (quartiles[2]-quartiles[0])

# query()를 사용해 값에서 벗어난 출생 수를 가진 행을 걸러줌
births = births.query('(births > @mu - 5 * @sig) & (births < @mu + 5 * @sig)')

births['day'] = births['day'].astype(int)

# 날짜, 월, 연도를 결합해 Date 인덱스를 생성
births.index = pd.to_datetime(10000 * births.year + 100 * births.month + births.day, format='%Y%m%d')
births['dayofweek'] = births.index.dayofweek

births

,year,month,day,gender,births,decade,dayofweek
1969-01-01,1969,1,1,F,4046,1960,2
1969-01-01,1969,1,1,M,4440,1960,2
1969-01-02,1969,1,2,F,4454,1960,3
1969-01-02,1969,1,2,M,4548,1960,3
1969-01-03,1969,1,3,F,4548,1960,4
...,...,...,...,...,...,...,...
1988-12-29,1988,12,29,M,5944,1980,3
1988-12-30,1988,12,30,F,5742,1980,4
1988-12-30,1988,12,30,M,6095,1980,4
1988-12-31,1988,12,31,F,4435,1980,5


In [41]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib

births.pivot_table('births', index='dayofweek', columns='decade', aggfunc='mean').plot()
plt.gca().set_xticklabels(['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])
plt.ylabel('mean births by day')

Using matplotlib backend: agg


Text(0, 0.5, 'mean births by day')

출생 수가 주중보다 주말에 약간 더 낮았다.

In [36]:
births_by_date = births.pivot_table('births', [births.index.month, births.index.day])
births_by_date.head()

births
1 1  4009.225
  2  4247.400
  3  4500.900
  4  4571.350
  5  4603.625

In [40]:
flg, ax = plt.subplots(figsize=(12,4))
births_by_date.plot(ax=ax)

# 분석
- 미국의 휴일(독립 기념일/ 노동절/ 추수감사절/ 크리스마스/ 새해 /..)에는 출생률이 급감했다
- 자연분만에 대한 깊은 정신적, 신체적 효과라기보다는 예정 분먼 맟 유도 분만을 선호했던 경향이 반영된 것으로 보임